In [ ]:
from glob import glob
from typing import Tuple, Callable, Dict
import PIL
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Resize, Normalize
import torchvision
import tests

In [ ]:
data_dir = 'processed_celeba_small/celeba/'

In [ ]:
def get_transforms(size):
    """ Transforms to apply to the image."""
    # TODO: edit this function by appening transforms to the below list
    transforms = [Resize(size),ToTensor(),Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]


    
    return Compose(transforms)

In [ ]:
import os

class DatasetDirectory(Dataset):
    def __init__(self, directory: str, transforms: Callable = None, extension: str = '.jpg'):
        self.directory=directory
        self.extension=extension
        self.transforms=transforms
    def __len__(self) -> int:
        count=0
        for files in os.listdir(self.directory):
            
            if files.endswith(self.extension):
                count+=1
        return count
    
        
    def __getitem__(self, index: int) -> torch.Tensor:
        """ load an image and apply transformation """
        filenameslist=[]
        for files in os.listdir(self.directory):
            if files.endswith(self.extension):
                filenameslist.append(files)
        name_to_read=self.directory+filenameslist[index]
        image=PIL.Image.open(name_to_read)
        function=get_transforms(64)
        tensor_image=function(image)
        
        return tensor_image
    

In [ ]:
    
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
# run this cell to verify your dataset implementation
dataset = DatasetDirectory(data_dir, get_transforms((64, 64)))
tests.check_dataset_outputs(dataset)

In [ ]:
dataset = DatasetDirectory(data_dir, get_transforms((64, 64)))
tests.check_dataset_outputs(dataset)

In [ ]:
def denormalize(images):
    """Transform images from [-1.0, 1.0] to [0, 255] and cast them to uint8."""
    return ((images + 1.) / 2. * 255).astype(np.uint8)

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(20, 4))
plot_size=20
for idx in np.arange(plot_size):
    ax = fig.add_subplot(2, int(plot_size/2), idx+1, xticks=[], yticks=[])
    img = dataset[idx].numpy()
    img = np.transpose(img, (1, 2, 0))
    img = denormalize(img)
    ax.imshow(img)

In [ ]:
from torch.nn import Module,Sequential, Flatten, Linear, LeakyReLU
import torch.nn as nn

In [1]:
class ConvBlock(nn.Module):
    """
    A convolutional block is made of 3 layers: Conv -> BatchNorm -> Activation.
    args:
    - in_channels: number of channels in the input to the conv layer
    - out_channels: number of filters in the conv layer
    - kernel_size: filter dimension of the conv layer
    - batch_norm: whether to use batch norm or not
    """
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, batch_norm: bool = True):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=2, padding=1, bias=False)
        self.batch_norm = batch_norm
        if self.batch_norm:
            self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.2)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        if self.batch_norm:
            x = self.bn(x)
        x = self.activation(x)
        return x

class Discriminator(Module):

    def __init__(self, conv_dim=32):
        super(Discriminator, self).__init__()

        # complete init function
        self.conv_dim = conv_dim

        
        self.conv1 = ConvBlock(3, conv_dim, 4, batch_norm=False) # first layer, no batch_norm
        
        self.conv2 = ConvBlock(conv_dim, conv_dim*2, 4)
        
        self.conv3 = ConvBlock(conv_dim*2, conv_dim*4, 4)
        self.conv4 = ConvBlock(conv_dim*4, conv_dim*8, 4)
        self.conv5 = ConvBlock(conv_dim*8, conv_dim*16, 4)
        self.conv6 = ConvBlock(conv_dim*16, 1, 4, batch_norm=False)
        
        #self.flatten = torch.nn.Flatten()
        # final, fully-connected layer
        self.fc = torch.nn.Sigmoid()

    def forward(self, x):
        # all hidden layers + leaky relu activation
        x = self.conv1(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        x = self.conv3(x)
        #print(x.shape)
        x = self.conv4(x)
        #print(x.shape)
        x = self.conv5(x)
        #print(x.shape)
        
        x = self.conv6(x)
        #print(x.shape)
        # final output layer
        x = self.fc(x)
        #x=torch.reshape(x,[-1,1,1,1])
        return x

NameError: name 'nn' is not defined

In [ ]:
discriminator = Discriminator()

tests.check_discriminator(discriminator)

In [ ]:
class DeconvBlock(Module):
    """
    A "de-convolutional" block is made of 3 layers: ConvTranspose -> BatchNorm -> Activation.
    args:
    - in_channels: number of channels in the input to the conv layer
    - out_channels: number of filters in the conv layer
    - kernel_size: filter dimension of the conv layer
    - stride: stride of the conv layer
    - padding: padding of the conv layer
    - batch_norm: whether to use batch norm or not
    """
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 kernel_size: int, 
                 stride: int,
                 padding: int,
                 batch_norm: bool = False):
        super(DeconvBlock, self).__init__()
        self.deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.batch_norm = batch_norm
        if self.batch_norm:
            self.bn = torch.nn.BatchNorm2d(out_channels)
        self.activation = torch.nn.ReLU()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.deconv(x)
        if self.batch_norm:
            x = self.bn(x)
        x = self.activation(x)
        return x


class Generator(nn.Module):
    """
    The generator model adapted from DCGAN
    args:
    - latent_dim: dimension of the latent vector
    - conv_dim: control the number of filters in the convtranspose layers
    """
    def __init__(self, latent_dim: int, conv_dim: int = 64):
        super(Generator, self).__init__()
        # transpose conv layers
        self.deconv1 = DeconvBlock(in_channels=latent_dim, 
                                   out_channels=conv_dim*8, kernel_size=4, 
                                   stride=1, padding=0)
        self.deconv2 = DeconvBlock(conv_dim*8, conv_dim*4, 4, 2, 1)
        
        self.deconv3 = DeconvBlock(conv_dim*4, conv_dim*2, 4, 2, 1)
        
        self.deconv4 = DeconvBlock(conv_dim*2, conv_dim*1, 4, 2, 1)
        
        
        
        self.deconv5 = nn.ConvTranspose2d(conv_dim, 3, 4, stride=2, padding=1)
        self.last_activation = nn.Tanh()
        
    def forward(self, x):
        x = self.deconv1(x)
        
        x = self.deconv2(x)
        
        x = self.deconv3(x)
        
        x = self.deconv4(x)
        
        x=self.deconv5(x)
        
        
        return x

In [ ]:
# run this cell to verify your generator implementation
latent_dim = 128
generator = Generator(latent_dim)
tests.check_generator(generator, latent_dim)

In [ ]:
train_on_gpu = torch.cuda.is_available()

In [ ]:
import torch.optim as optim

lr = 0.0002
beta1=0.5
beta2=0.999 # default value
def create_optimizers(generator: Module, discriminator: Module,lr,beta1,beta2):
    g_optimizer = optim.Adam(generator.parameters(), lr, [beta1,beta2])
    d_optimizer =optim.Adam(discriminator.parameters(), lr, [beta1,beta2])
    return g_optimizer, d_optimizer


def real_loss(D_out, smooth=False):
    batch_size = D_out.size(0)
    labels = torch.ones(batch_size).cuda()
    criterion = nn.BCELoss()
    loss = criterion(D_out.squeeze().cuda(), labels)
    return loss

def fake_loss(D_out):
    batch_size = D_out.size(0)
    labels = torch.zeros(batch_size).cuda() # fake labels = 0
    criterion_fake_loss = nn.BCELoss()
    loss = criterion_fake_loss(D_out.squeeze().cuda(), labels)
    return loss

def generator_loss(fake_logits):
    return real_loss(fake_logits)

def discriminator_loss(real_logits,fake_logits):
    return fake_loss(fake_logits)+real_loss(real_logits)

In [ ]:
import torch.optim as optim

In [ ]:
def gradient_penalty(discriminator, real_samples, fake_samples):
    """ This function enforces """
    gp = 0
    # TODO (Optional): implement the gradient penalty
    return gp

In [ ]:
def generator_step(batch_size: int, latent_dim: int):
    """ One training step of the generator. """
    noise=torch.randn(batch_size,latent_dim,1,1).cuda()
    fake_images=generator(noise)
    generator.zero_grad()
    fake_logits=discriminator(fake_images)
    g_loss=generator_loss(fake_logits)
    g_loss.backward()
    g_optimizer.step()
    return {'loss': g_loss,"fake_logits":fake_logits}
    #return g_loss


def discriminator_step(batch_size: int, latent_dim: int, real_images: torch.Tensor):
    """ One training step of the discriminator. """
    # TODO: implement the discriminator step (foward pass, loss calculation and backward pass)
    discriminator.zero_grad()
    real_logits=discriminator(real_images)
    fake_images=generator(torch.randn(batch_size,latent_dim,1,1).cuda())
    fake_logits=discriminator(fake_images.detach()) 
    d_loss=discriminator_loss(real_logits,fake_logits)
    d_loss.backward()
    d_optimizer.step()
    
    return {'loss': d_loss}
    

In [ ]:
from datetime import datetime
latent_dim = 128
device = 'cuda'
n_epochs = 10
batch_size = 16

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
print_every = 10

# Create optimizers for the discriminator D and generator G
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)


g_optimizer, d_optimizer = create_optimizers(generator, discriminator,lr=0.0002,beta1=0.5,beta2=0.999)

dataloader = DataLoader(dataset, 
                        batch_size=64, 
                        shuffle=True, 
                        num_workers=4, 
                        drop_last=True,
                        pin_memory=False)

In [ ]:
# In[27]:


"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""

def display(fixed_latent_vector: torch.Tensor):
    """ helper function to display images during training """
    fig = plt.figure(figsize=(14, 4))
    plot_size = 16
    for idx in np.arange(plot_size):
        ax = fig.add_subplot(2, int(plot_size/2), idx+1, xticks=[], yticks=[])
        img = fixed_latent_vector[idx, ...].detach().cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        img = denormalize(img)
        ax.imshow(img)
    plt.show()

In [ ]:
fixed_latent_vector = torch.randn(16, latent_dim, 1, 1).float().cuda()

losses = []
for epoch in range(n_epochs):
    for batch_i, real_images in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
     
        ####################################
        g_loss=generator_step(batch_size,latent_dim)
        
        d_loss=discriminator_step(batch_size,latent_dim,real_images)
        ####################################
        
        if batch_i % print_every == 0:
            # append discriminator loss and generator loss
            d = d_loss['loss'].item()
            g = g_loss['loss'].item()
            losses.append((d, g))
            # print discriminator and generator loss
            time = str(datetime.now()).split('.')[0]
            print(f'{time} | Epoch [{epoch+1}/{n_epochs}] | Batch {batch_i}/{len(dataloader)} | d_loss: {d:.4f} | g_loss: {g:.4f}')
    
    # display images during training
    generator.eval()
    generated_images = generator(fixed_latent_vector)
    display(generated_images)
    generator.train()

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator', alpha=0.5)
plt.plot(losses.T[1], label='Generator', alpha=0.5)
plt.title("Training Losses")
plt.legend()

In [ ]:
# In[52]:


nn.BCE


# ### Question: What do you notice about your generated samples and how might you improve this model?
# When you answer this question, consider the following factors:
# * The dataset is biased; it is made of "celebrity" faces that are mostly white
# * Model size; larger models have the opportunity to learn more features in a data feature space
# * Optimization strategy; optimizers and number of epochs affect your final result
# * Loss functions

# **Answer:** (Write your answer in this cell)

# ### Submitting This Project
# When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_face_generation.ipynb".  
# 
# Submit the notebook using the ***SUBMIT*** button in the bottom right corner of the Project Workspace.

# In[ ]:


loss=torch.nn.BCEWithLogitsLoss()
loss(torch.Tensor([-1]),torch.Tensor([1]))